<a href="https://colab.research.google.com/github/darshbaxi/Tensorflow-Practice/blob/main/churn_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner
from tensorflow import keras as kr 
from tensorflow.keras import layers


In [3]:
df=pd.read_csv('Churn_Modelling.csv')

In [4]:
df.shape


(10000, 14)

In [5]:
y=df.iloc[:,-1]

In [6]:
geo=pd.get_dummies(df['Geography'])

In [7]:
gen=pd.get_dummies(df['Gender'],drop_first=True)

In [8]:
df=pd.concat([df,geo,gen],axis=1)

In [9]:
df=df.drop('Geography',axis=1)
df=df.drop('Gender',axis=1)
df=df.drop('RowNumber',axis=1)
df=df.drop('Surname',axis=1)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,Male
0,15634602,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,0
1,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,0
2,15619304,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,0
3,15701354,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,0
4,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15606229,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,1
9996,15569892,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,1
9997,15584532,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,0
9998,15682355,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,1


In [12]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x.shape

(10000, 13)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [14]:

def build_model(hp):
  model=kr.Sequential()
  for i in range(hp.Int('num_layer',1,5)):
    model.add(layers.Dense(units=hp.Int('units'+str(i),min_value=32,max_value=512,step=32),activation='relu'))
  model.add(layers.Dense(units=1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=kr.optimizers.Adam(hp.Choice('learning_rate',[1e-4,1e-3,1e-2])),metrics=['accuracy'])
  return model  


In [15]:
# model.add(layers.Dense(units=6,kernel_initializer='he_uniform',activation='relu',input_dim=13))
# model.add(layers.Dense(units=6,kernel_initializer='he_uniform',activation='relu'))
# model.add(layers.Dense(units=4,kernel_initializer='he_uniform',activation='relu'))
# model.add(layers.Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))




In [22]:

tuner=keras_tuner.RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='project1',
    project_name='churn_model'
)

In [23]:
tuner.search(x_train, y_train,batch_size=10,epochs=5)

Trial 3 Complete [00h 00m 44s]
accuracy: 0.5476119518280029

Best accuracy So Far: 0.5476119518280029
Total elapsed time: 00h 00m 44s
